In [1]:
from selenium import webdriver
import pandas as pd
import openpyxl as op
import time
import re
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
get_number = re.compile('.*(?=\n)')
get_percent = re.compile('.*(?=%)')
get_url_without_slash = re.compile(".*[0-9a-z]")
password = !echo %password%

In [2]:
def reset():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")

def login():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")
    #아이디입력
    driver.find_element_by_css_selector('#Email').send_keys('taehun9312@gmail.com')
    driver.find_element_by_css_selector('#next').click()
    time.sleep(1)
    #비밀번호입력
    driver.find_element_by_css_selector('#Passwd').send_keys(password[0])
    driver.find_element_by_css_selector('#signIn').click()
    
def login2():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")
    #아이디입력
    driver.find_element_by_css_selector('#identifierId').send_keys('taehun9312@gmail.com')
    driver.find_element_by_css_selector('.RveJvd.snByac').click()
    time.sleep(1)
    #비밀번호입력
    driver.find_element_by_css_selector('.whsOnd.zHQkBf').send_keys(password[0])
    driver.find_element_by_css_selector('.RveJvd.snByac').click()
    
def s_to_i(value):
    value = value.replace(",","")
    value = int(value)
    return value

def check(compared_value):
    if type(compared_value) == list:
        for value in compared_value:
            if value in global_value: raise ValueError
    elif compared_value in global_value: raise ValueError
    else: print("No_duplication")
        
def get_pure_url(url):
    splited_url = url.split("/%")
    url = splited_url[0]
    url = get_url_without_slash.findall(url)[0]
    return url

def get_number_item(unprocessed_number):
    processed_number = unprocessed_number.split("(")[0]
    return s_to_i(processed_number)

        
def calculate_date(origin_date, interval, multiple_number):
    date_set = _get_date(origin_date)
    d1 = date(date_set["year"], date_set["month"], date_set["day"])
    if interval == "month":
        d2 = d1 + relativedelta(months=+1*multiple_number)
    else:
        d2 = d1 + timedelta(days = interval*multiple_number)
    next_date = _to_date(d2)
    return next_date

def _get_date(date):
    date = str(date)
    year = "20" + date[0:2]
    month = date[2:4]
    day = date[4:6]
    if month[0] == "0":
        month = month.replace("0","")
    return {"year": int(year), "month": int(month), "day": int(day)}

def _to_date(date):
    year = str(date.year)[2:4]
    if date.month < 10:
        month = "0" + str(date.month)
    else:
        month = str(date.month)
    if date.day < 10:
        day = "0" + str(date.day)
    else:    
        day = str(date.day)
    return year+month+day

In [12]:
pd.DataFrame?

In [17]:
a = pd.DataFrame(columns=["a", "b"], data = d)

In [9]:
class GA:
    
    def __init__(self, url):
        global start
        global end 
        self.df = pd.DataFrame()
        self.date = "/%3F_u.date00%3D20{start}%26_u.date01%3D20{end}".format(start= start, end=end)
        self.original_url = url
        self.url = get_pure_url(url) + self.date
        self.start = start
        self.end = end
        self.retry = 40
        self.days = 1
        self.df_start = str(start)
        self.df_end = str(end)
        self.dic_values = {}
        self.seg = "0"
        driver.get(self.url)
        
    def set_date(self, interval, number):
        self.interval = interval
        self.days = number

    def set_segment_url(self, seg, keep):
        if keep == "on": self.seg = seg
        self.url = get_url_without_slash.findall(self.url)[0] + seg
        self.df = self.df.transpose()
        #"%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ"
        self.go_changed_url(0)
        
    def set_tab(self, n_tab):
        self.tab = n_tab
        
    def set_value(self, n_value):
        if type(n_value) != list:
            self.value = [n_value]
        else:
            self.value = n_value
        
    def set_retry(self, n):
        self.retry = n
    
    def go_changed_url(self, n_try):
        start = calculate_date(self.start, self.interval, n_try)
        end = calculate_date(self.end, self.interval, n_try)
        self.df_start = str(start)
        self.df_end = str(end)
        self.date =  "/%3F_u.date00%3D20{start}%26_u.date01%3D20{end}".format(start= start, end=end)
        if len(self.seg) > 1:
            self.url = get_pure_url(self.original_url) + self.date + self.seg
        else:
            self.url = get_pure_url(self.original_url) + self.date
        driver.get(self.url)
        
    def get(self, numbers):
        for n_try in range(1, self.days+1):
            if type(numbers) == int:
                value = self._come_to_me(numbers)
                return value
            else:
                for n in numbers:
                    self.dic_values = self._fill_dic_values(n, self.dic_values)
            for title, value in self.dic_values.items():
                for n in range(0, 10):
                    try:
                        a = self.df.loc[self.df_start+" - "+self.df_end, title]
                        if str(a) == 'nan': raise KeyError
                    except KeyError:
                        if n == 0:
                            self.df.loc[self.df_start+" - "+self.df_end, title] = value
                        else:
                            title = title + str(n)
                            self.df.loc[self.df_start+" - "+self.df_end, title] = value
                        break
                    else:
                        continue
                        
                        
            if self.days > 1 and n_try < self.days: 
                    self.dic_values = {}
                    self.go_changed_url(n_try)  
        self.df = self.df.transpose()
 

    def _fill_dic_values(self, numbers, dic_values):
            n = numbers
            value = self._come_to_me(n)
            titles = self._get_title(self.value[n-1])
            
            if type(value) != list:
                if type(self.value[n-1]) != list:
                    #title 1, value 1
                    title = titles[0]
#                     if title in dic_values:
#                         title = title + str(n)
                    dic_values[title] = value
                else:
                    #titile more, value 1
                    nth = self.value[n-1][1]
                    title = titles[nth-1]
#                     if title in dic_values:
#                         title = title + str(n)
                    dic_values[title] = value
            else:
                #title more, value more
                for i, v in enumerate(value):
                    title = titles[i]
#                     if title in dic_values:
#                         title = title + str(n)
                    dic_values[title] = v 
            return dic_values
    
    def _come_to_me(self, n):
        global global_value
        error_check = "on"
        self._click_tab(n)
        value_spec = self.value[n-1]

        if type(value_spec) == list: 
            number = value_spec[0] 
            nth = value_spec[1]
        else: 
            number = value_spec
            nth = value_spec

        if type(value_spec) == list or value_spec == 1:
            value = self._get_value(nth, error_check)
            error_check = "off"
            value_for_compare = self._get_values(number, error_check)
        else:
            value_for_compare = self._get_values(number, error_check)
            value = value_for_compare
        global_value = value_for_compare
        return value

    def _get_title(self, n):
        arr_title = []
        unprocessed_titles = driver.find_elements_by_css_selector("._GArO")
        for title in unprocessed_titles:
            arr_title.append(title.text)
        return arr_title
    
    def get_percent(self, n):
        arr_value = driver.find_elements_by_css_selector("._GAj8")
        percent = arr_value[n-1].text
        processed_percent = float(get_percent.findall(percent)[0])
        print(processed_percent)
        return processed_percent
        
            
#     def get_item(self):
#         for i in range(0, self.retry):
#             try:
#                 #Screen name, Event lbael.. etc
#                 target_name = driver.find_elements_by_css_selector("._GAkub")[0].text
#                 #Unique screen view, screen view.. etc
#                 table_heads = driver.find_elements_by_css_selector("._GAYh.ACTION-sort")
#                 arr_table_heads = []
#                 for table_head in table_heads:
#                     arr_table_heads.append(table_head.text)
#                 num_cols = len(arr_table_heads)
#                 arr_table_heads.insert(0,target_name)
#                 #home_banner_1, home_banner_2.. etc
#                 target_list = driver.find_elements_by_css_selector("._GApu")
#                 #700, 600.. etc
#                 dataset = driver.find_elements_by_css_selector("._GAtjb")
#                 self.df = pd.DataFrame(columns = arr_table_heads)
#                 for row_num, target in enumerate(target_list):
#                     self.df[target_name, target] =
#                         for i in range(0, num_cols+1):
#                             df.loc[item,dataset[row_num*num_cols+i]
                        
#                         value = get_number_item(dataset[i*2+nth-1].text)
#                         check(value)
#                         print("It's ok")
#                         arr_data = []
#                         for data in dataset:
#                             arr_data.append(data.text)
#                         global_value = arr_data
#                         print(value)
#                         return value
#                 time.sleep(1)
#             except:
#                 time.sleep(4)
#                 print("It's not ok")
#                 continue
                        

    #n번째 값 가져오기
    def _get_value(self, n, error_check):
        for i in range(0, self.retry):
            try:
                print(i)
                value = driver.find_element_by_css_selector(".ID-stage{n}-top._GAhFb > ._GAXm > ._GAoCb".format(n = n)).text
                if n != 1: value = get_number.findall(value)[0]
                value = s_to_i(value)
                print("value is", value)
                if error_check == "on":
                    check(value)
                print("It's ok")
                break
            except:
                time.sleep(4)
                print("It's not ok")
                continue
        print(value)
        return value
    
    #n개의 값 가져오기
    def _get_values(self, range_n, error_check):
        arr_value = []
        for n in range(1, range_n+1):
            value = self._get_value(n, error_check)
            arr_value.append(value)
        print(arr_value)
        return arr_value

    
    #커스텀 리포트 내 탭
    def _click_tab(self, n):
        if n == 1:
            driver.get(self.url)
        else:
            for i in range(0, self.retry):
                try:
                    driver.find_elements_by_css_selector("._GAit._GAl")[n-1].click()
                    break
                except:

                    time.sleep(3)
                    print("It retry to click")
                    continue

In [4]:
# tab_category = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/C02IOMc8QBmCo6k8wf6aGQ/a10587421w80067081p102782120/%3F_u.date00%3D20170424%26_u.date01%3D20170425%26_.useg%3Dbuiltin13%26_r.tabId%3D3915/")
# tab_category.set_tab(5)
# tab_category.set_value([3,3,4,4])
# tab_category.set_date(1,30)
# tab_category.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "on")
# tab_category.get([1,2,3,4])

In [5]:
# excel = op.load_workbook(r"C:\Users\joseph_th\Desktop\Dashboard\Dashboard_for_GA.xlsx")
# s1 = excel["여행 Traffic overview"]["AC"]
# s2 = excel["여행 딜"]["W"]
# s3 = excel["실시간항공"]["Z"]
# s4 = excel["실시간호텔"]["U"]
# s5 = excel["APP 홈 기여"]["S"]
# excel.save(r"C:\Users\joseph_th\Desktop\Dashboard\Dashboard_for_GA_result.xlsx")

In [5]:
driver = webdriver.Chrome(executable_path=r"C:\Users\joseph_th\Downloads\chromedriver_win32\chromedriver")
login2()

In [22]:
global_value = []
start = 170608
end = 170608
reset()

In [5]:
# html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")
# bsObj = BeautifulSoup(html, "html.parser")
# for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")):
#     if "href" in link.attrs:
#         print(link.attrs["href"])

In [17]:
Tab_purchase.df

,170605 - 170605,170606 - 170606
여행탭,21731.0,21731.0
구매완료,258.0,198.0


In [10]:
Tab_purchase = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/TBEAoU8fTnGAoFObTeqQdQ/a10587421w80067081p102782120/%3F_u.date00%3D20170410%26_u.date01%3D20170410%26_.useg%3DuserUILGqtAkQ6WoEA3NVyENtA/")
Tab_purchase.set_tab(1)
Tab_purchase.set_value([[2,1]])
Tab_purchase.set_date(1,2)
Tab_purchase.get([1])

Tab_purchase.set_segment_url("%26_.useg%3DuserUILGqtAkQ6WoEA3NVyENtA", "on")
Tab_purchase.set_value([[2,2]])
Tab_purchase.get([1])

0
It's not ok
1
value is 18502
No_duplication
It's ok
18502
0
value is 18502
It's ok
18502
0
value is 1891
It's ok
1891
[18502, 1891]
0
value is 18502
It's not ok
1
value is 21731
No_duplication
It's ok
21731
0
value is 21731
It's ok
21731
0
value is 2471
It's ok
2471
[21731, 2471]
0
value is 258
No_duplication
It's ok
258
0
value is 320
It's ok
320
0
value is 258
It's ok
258
[320, 258]
0
value is 258
It's not ok
1
value is 258
It's not ok
2
value is 198
No_duplication
It's ok
198
0
value is 252
It's ok
252
0
value is 198
It's ok
198
[252, 198]


In [ ]:
#여행홈 이용률_And 
Tour_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/BNAhfOhCRYGSL64VQWI9pQ/a10587421w80067081p102782120/%3F_r.tabId%3D1169/")
Tour_1.set_tab(5)
Tour_1.set_value([2,[2,2],[2,2],[2,2],[2,2]])
Tour_1.set_date(1,5)
# Tour_1.set_segment_url("%26_.useg%3Duser3pWJ7sIwS-mDV3x54DAbeg", "on") #4.1.9v ↑
Tour_1.set_segment_url("%26_.useg%3DuseryUAfuE0oTlOfVJjkBz-Mcw", "on") #4.2.2v ↑
Tour_1.get([1,2,3,4])
# Tour_1.set_segment_url("%26_.useg%3DuseryUAfuE0oTlOfVJjkBz-Mcw", "on") #4.2.2v ↑
# Tour_1.get([1,5])

Tour_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hT5jydKhQni5if6X7sW0rQ/a10587421w80067081p102782120/")
Tour_2.set_tab(4)
Tour_2.set_value([[2,2],[2,2],[2,2],[2,2]])
Tour_2.set_date(1,5)
# Tour_2.set_segment_url("%26_.useg%3Duser3pWJ7sIwS-mDV3x54DAbeg", "on")
Tour_1.set_segment_url("%26_.useg%3DuseryUAfuE0oTlOfVJjkBz-Mcw", "on") #4.2.2v ↑
Tour_2.get([1,2,3,4])

0
It's not ok
1
value is 18586
No_duplication
It's ok
18586
0
value is 3817
No_duplication
It's ok
3817
[18586, 3817]
0
value is 3817
It's not ok
1
value is 7537
No_duplication
It's ok
7537
0
value is 18586
It's ok
18586
0
value is 7537
It's ok
7537
[18586, 7537]
0
value is 7537
It's not ok
1
value is 7537
It's not ok
2
value is 2182
No_duplication
It's ok
2182
0
value is 18586
It's ok
18586
0
value is 2182
It's ok
2182
[18586, 2182]
0
value is 2182
It's not ok
1
value is 694
No_duplication
It's ok
694
0
value is 18586
It's ok
18586
0
value is 694
It's ok
694
[18586, 694]
0
value is 18586
It's not ok
1
value is 16742
No_duplication
It's ok
16742
0
value is 3276
No_duplication
It's ok
3276
[16742, 3276]
0
value is 3276
It's not ok
1
value is 6507
No_duplication
It's ok
6507
0
value is 16742
It's ok
16742
0
value is 6507
It's ok
6507
[16742, 6507]
0
value is 6507
It's not ok
1
value is 6507
It's not ok
2
value is 1944
No_duplication
It's ok
1944
0
value is 16742
It's ok
16742
0
value is 

In [14]:
Tour_1.df

,170608 - 170608,170609 - 170609,170610 - 170610,170611 - 170611,170612 - 170612
여행 홈,21544.0,19095.0,21339.0,22321.0,23732.0
1. 카테고리 클릭,4492.0,3825.0,3748.0,4265.0,5074.0
아무거나 클릭,8713.0,7458.0,7586.0,8463.0,9873.0
"3. 실시간 항공, 호텔 검색클릭",2470.0,2174.0,2037.0,2269.0,3077.0
4. 기획전 배너 클릭,796.0,630.0,677.0,785.0,820.0
5. 개인화 영역 클릭,1028.0,848.0,995.0,1081.0,1154.0


In [16]:
Tour_1.df

,170608 - 170608,170609 - 170609,170610 - 170610,170611 - 170611,170612 - 170612
여행 홈,18586.0,16742.0,18926.0,20044.0,21442.0
1. 카테고리 클릭,3817.0,3276.0,3257.0,3745.0,4469.0
5. 개인화 영역 클릭,1028.0,848.0,995.0,1081.0,1154.0


In [12]:
Tour_2.df

,170608 - 170608,170609 - 170609,170610 - 170610,170611 - 170611,170612 - 170612
테마 컨텐츠,421.0,433.0,564.0,597.0,482.0
운영 컨텐츠,1128.0,672.0,727.0,785.0,1134.0
할인 호텔,81.0,76.0,83.0,97.0,104.0
지역/나라딜,949.0,757.0,918.0,1024.0,1069.0


In [27]:
pd.concat([Tour_1.df, Tour_2.df])

,170602 - 170602,170603 - 170603,170604 - 170604,170605 - 170605,170606 - 170606
여행 홈,23184.0,11469.0,14507.0,16123.0,19462.0
1. 카테고리 클릭,4521.0,2289.0,2988.0,3314.0,3630.0
"3. 실시간 항공, 호텔 검색클릭",2489.0,1973.0,2125.0,2386.0,2489.0
4. 기획전 배너 클릭,745.0,516.0,629.0,810.0,745.0
여행 홈1,2641.0,NaN,NaN,NaN,NaN
1. 카테고리 클릭1,582.0,NaN,NaN,NaN,NaN
5. 개인화 영역 클릭,140.0,633.0,810.0,886.0,1028.0
테마 컨텐츠,NaN,NaN,NaN,478.0,577.0
운영 컨텐츠,NaN,NaN,NaN,1213.0,1240.0
할인 호텔,NaN,NaN,NaN,80.0,82.0


In [14]:
#국내, 해외, 항공호텔, 전체카테고리
And_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/XAzFnmCtTl6SM1Cmc9OVNw/a10587421w80067081p102782120/")
And_1.set_tab(5)
And_1.set_value([1,1,1,1])
And_1.get([1,2,3,4])

#티몬 전체 카테고리
And_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/gmdw7utOS6OyMdXRs_jizg/a10587421w80067081p102782120/")
And_2.set_tab(1)
And_2.set_value(1)
And_2.get([1])

# #국내, 해외, 여행전체 딜
# And_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/c9agY1srRx2PaD8JAkK0yw/a10587421w80067081p102782120/")
# And_3.set_tab(3)
# And_3.set_value([3,3,3])
# And_3.get([1,2,3])


# And_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "nope")
# And_3.set_value([[3,3],[3,3],[3,3]])
# And_3.get([1,2,3])


#항공검색 해외, 국내, 해외->호텔메인, 해외->국내항공메인
And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5fyYOU-QTN6tQSwRlObkUw/a10587421w80067081p102782120/%3F_u.date00%3D20170101%26_u.date01%3D20170301%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
And_4.set_tab(4)
And_4.set_value([2,2,[2,2],[2,2]])
And_4.get([1,2,3,4])

#항공예약 클릭 국내, 해외 
And_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/_yDrAZzXTaeMOGlYkwqqbw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D%26_r.tabId%3D6028/")
And_5.set_tab(2)
And_5.set_value([1,1])
And_5.get([1,2])

#B.com 카테고리/시스템 큐레이션
And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_6.set_tab(5)
And_6.set_value([2,1,1,2,1])
And_6.get([1,2,3,4,5])

#B.com 매뉴얼 큐레이션/이벤트
And_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/qWyiPek-Tsqgkok-rZsIjA/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_7.set_tab(2)
And_7.set_value([2,2])
And_7.get([1,2])

#홈 -> 딜상세 국내, 해외, 카테고리숏컷
And_8 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/4rkbjB7zTGmgcRAlJaVzrg/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_8.set_tab(3)
And_8.set_value([2,[2,2],1])
And_8.get([1,2,3])

#홈 -> 콜렉션(국내), 해외
And_9 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/lUOk-ms-R966phrt04_yfA/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_9.set_tab(2)
And_9.set_value([2,[2,2]])
And_9.get([1,2])


#여행탭 -> 카테고리, 항공검색결과
And_10 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/rUsbr2MuTou-MkNzI27mfw/a10587421w80067081p102782120/")
And_10.set_tab(3)
And_10.set_value([2,[2,2]])
And_10.get([1,2])

0
It's not ok
1
value is 118030
No_duplication
It's ok
118030
0
value is 118030
It's ok
118030
[118030]
0
value is 118030
It's not ok
1
value is 50112
No_duplication
It's ok
50112
0
value is 50112
It's ok
50112
[50112]
0
value is 50112
It's not ok
1
value is 33097
No_duplication
It's ok
33097
0
value is 33097
It's ok
33097
[33097]
0
value is 33097
It's not ok
1
value is 184107
No_duplication
It's ok
184107
0
value is 184107
It's ok
184107
[184107]
0
value is 184107
It's not ok
1
value is 681973
No_duplication
It's ok
681973
0
value is 681973
It's ok
681973
[681973]
0
value is 681973
It's not ok
1
value is 681973
It's not ok
2
value is 27649
No_duplication
It's ok
27649
0
value is 11901
No_duplication
It's ok
11901
[27649, 11901]
0
value is 27649
It's not ok
1
value is 27649
It's not ok
2
value is 10646
No_duplication
It's ok
10646
0
value is 8640
No_duplication
It's ok
8640
[10646, 8640]
0
value is 8640
It's not ok
1
value is 3664
No_duplication
It's ok
3664
0
value is 27649
It's ok
27

In [15]:
#국내, 해외, 항공호텔, 전체카테고리
iOS_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/f6fCST7zQwalCMN1dpSuPw/a10587421w80067081p88823264/")
iOS_1.set_tab(5)
iOS_1.set_value([1,1,1,1])
iOS_1.get([1,2,3,4])


#티몬 전체 카테고리
iOS_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p29BV_8iRc2OovJCvhcg_Q/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402/")
iOS_2.set_tab(1)
iOS_2.set_value(1)
iOS_2.get([1])

# #국내딜, 해외딜, 전체딜
# iOS_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/A9m3wyZgQTmxJrO3lQ6UMw/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402%26_r.tabId%3D4646%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# iOS_3.set_tab(3)
# iOS_3.set_value([3,3,3])
# iOS_3.get([1,2,3])

# iOS_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "nope")
# iOS_3.set_value([[3,3],[3,3],[3,3]])
# iOS_3.get([1,2,3])

#항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/2K60xdJeT2isKsPe4tgpEQ/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170320/")
iOS_4.set_tab(4)
iOS_4.set_value([2,2,[2,2],[2,2]])
iOS_4.get([1,2,3,4])

#항공예약클릭 국내, 해외
iOS_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/btj6gUJmQ2-NZsSB1vB3tw/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170331%26_.useg%3D/")
iOS_5.set_tab(2)
iOS_5.set_value([1,1])
iOS_5.get([1,2])

#B.com 카테고리/시스템 큐레이션
iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/mzJgxll5SXqATSN_oHrUiQ/a10587421w80067081p88823264/")
iOS_6.set_tab(5)
iOS_6.set_value([2,1,1,2,1])
iOS_6.get([1,2,3,4,5])

#B.com 매뉴얼 큐레이션/이벤트
iOS_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/tZRMJC4JRhm_X9Ql6C4KOg/a10587421w80067081p88823264/")
iOS_7.set_tab(2)
iOS_7.set_value([2,2])
iOS_7.get([1,2])

#홈 -> 딜상세(국내), 해외, 여행카테고리숏컷
iOS_8 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/GDfZCJv9RSC7dYyccZZ1lA/a10587421w80067081p88823264/")
iOS_8.set_tab(3)
iOS_8.set_value([2,[2,2],1])
iOS_8.get([1,2,3])

#홈 -> 콜렉션(국내), 해외
iOS_9 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/9rdCrGeYTaCIuaCGnfLoRQ/a10587421w80067081p88823264/")
iOS_9.set_tab(2)
iOS_9.set_value([2,[2,2]])
iOS_9.get([1,2])

#여행홈 -> 
iOS_10 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/f30PcWxBRsWzVzv0emSDdw/a10587421w80067081p88823264/")
iOS_10.set_tab(3)
iOS_10.set_value([2,[2,2]])
iOS_10.get([1,2])

0
value is 136286
It's not ok
1
value is 47440
No_duplication
It's ok
47440
0
value is 47440
It's ok
47440
[47440]
0
value is 47440
It's not ok
1
value is 22654
No_duplication
It's ok
22654
0
value is 22654
It's ok
22654
[22654]
0
value is 22654
It's not ok
1
value is 20522
No_duplication
It's ok
20522
0
value is 20522
It's ok
20522
[20522]
0
value is 20522
It's not ok
1
value is 81090
No_duplication
It's ok
81090
0
value is 81090
It's ok
81090
[81090]
0
value is 81090
It's not ok
1
value is 350300
No_duplication
It's ok
350300
0
value is 350300
It's ok
350300
[350300]
0
value is 350300
It's not ok
1
value is 17315
No_duplication
It's ok
17315
0
value is 8032
No_duplication
It's ok
8032
[17315, 8032]
0
value is 17315
It's not ok
1
value is 6209
No_duplication
It's ok
6209
0
value is 5075
No_duplication
It's ok
5075
[6209, 5075]
0
value is 5075
It's not ok
1
value is 2421
No_duplication
It's ok
2421
0
value is 17315
It's ok
17315
0
value is 2421
It's ok
2421
[17315, 2421]
0
value is 242

In [16]:
#국내카테고리, 해외, 항공호텔, 전체
Mo_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/V-T6iNbTQWi5USC2dSi5tA/a10587421w80067081p89357101/")
Mo_1.set_tab(5)
Mo_1.set_value([1,1,1,2])
Mo_1.get([1,2,3,4])

#티몬 홈 전체
Mo_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/zoovm6xETN-Cd6JF9s2_hQ/a10587421w80067081p89357101/")
Mo_2.set_tab(1)
Mo_2.set_value(1)
Mo_2.get([1])

# #여행딜 전체, 국내, 해외
# Mo_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/upJUz96ERVGUIE2WWd8Tgg/a10587421w80067081p89357101/")
# Mo_3.set_tab(3)
# Mo_3.set_value([3,3,3])
# Mo_3.get([1,2,3])
# Mo_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "nope")
# Mo_3.set_value([[3,3],[3,3],[3,3]])
# Mo_3.get([1,2,3])

#항공검색 해외, 국내, 해외->호텔, 해외->국내
Mo_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/7aMTsZECQu2GVnxwp3IXRg/a10587421w80067081p89357101/")
Mo_4.set_tab(4)
Mo_4.set_value([2,2,[2,2],[2,2]])
Mo_4.get([1,2,3,4])

#항공예약클릭 국내, 해외
Mo_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Shs74NpmR1Wcy5isHt_ZGA/a10587421w80067081p89357101/")
Mo_5.set_tab(2)
Mo_5.set_value([1,1])
Mo_5.get([1,2])

#부킹닷컴 큐레이션, 호텔메인, 이벤트
Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/")
Mo_6.set_tab(3)
Mo_6.set_value([2,2,2])
Mo_6.get([1,2,3])

#  홈화면 -> 딜상세
Mo_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/tbyK5vXiTnuyfCcVRNtBJQ/a10587421w80067081p89357101/")
Mo_7.set_tab(2)
Mo_7.set_value([2,[2,2]])
Mo_7.get([1,2])

0
value is 52887
It's not ok
1
value is 27019
No_duplication
It's ok
27019
0
value is 27019
It's ok
27019
[27019]
0
value is 27019
It's not ok
1
value is 9480
No_duplication
It's ok
9480
0
value is 9480
It's ok
9480
[9480]
0
value is 9480
It's not ok
1
value is 9527
No_duplication
It's ok
9527
0
value is 9527
It's ok
9527
[9527]
0
value is 9527
It's not ok
1
value is 30924
No_duplication
It's ok
30924
0
value is 18270
No_duplication
It's ok
18270
[30924, 18270]
0
value is 30924
It's not ok
1
value is 232482
No_duplication
It's ok
232482
0
value is 232482
It's ok
232482
[232482]
0
value is 232482
It's not ok
1
value is 8164
No_duplication
It's ok
8164
0
value is 2318
No_duplication
It's ok
2318
[8164, 2318]
0
value is 8164
It's not ok
1
value is 2862
No_duplication
It's ok
2862
0
value is 1426
No_duplication
It's ok
1426
[2862, 1426]
0
value is 1426
It's not ok
1
value is 841
No_duplication
It's ok
841
0
value is 8164
It's ok
8164
0
value is 841
It's ok
841
[8164, 841]
0
value is 841
It

In [17]:
#카테고리 국내, 해외, 항공호텔, 전체
PC_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/RFQ6OlJDT5-6MMKnjtJkcA/a10587421w80067081p84826225/")
PC_1.set_tab(2)
PC_1.set_value([1,1])
PC_1.get([1,2])

# 티몬 전체 카테고리
PC_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ZcNefO8cTRe75IKFbPcCWw/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_2.set_tab(1)
PC_2.set_value(1)
PC_2.get([1])

# # 여행 전체, 국내, 해외딜
# PC_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hRoEL4w4RCyYpcmhDbvgWw/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_3.set_tab(3)
# PC_3.set_value([3,3,3])
# PC_3.get([1,2,3])
# PC_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "nope")
# PC_3.set_value([[3,3],[3,3],[3,3]])
# PC_3.get([1,2,3])

# 항공검색 국내, 해외
PC_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/KkWuVj5-SCOH7ddtrf4bOA/a10587421w80067081p84826225/")
PC_4.set_tab(2)
PC_4.set_value([2,2])
PC_4.get([1,2])

# 예약클릭 국내, 해외
PC_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/YspUfeZLQ7KyhFzRxehRcQ/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_5.set_tab(2)
PC_5.set_value([1,1])
PC_5.get([1,2])

# B.com 카테고리/시스템/매뉴얼큐레이션/이벤트
PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_6.set_tab(4)
PC_6.set_value([1,2,2,2])
PC_6.get([1,2,3,4])

#  홈화면 -> 딜상세
PC_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/S50YB2xDSLCbPXt17xrXBw/a10587421w80067081p84826225/%3F_u.date00%3D20170501%26_u.date01%3D20170507/")
PC_7.set_tab(2)
PC_7.set_value([2,[2,2]])
PC_7.get([1,2])

0
value is 755056
It's not ok
1
value is 76970
No_duplication
It's ok
76970
0
value is 76970
It's ok
76970
[76970]
0
value is 76970
It's not ok
1
value is 35044
No_duplication
It's ok
35044
0
value is 35044
It's ok
35044
[35044]
0
value is 35044
It's not ok
1
value is 373048
No_duplication
It's ok
373048
0
value is 373048
It's ok
373048
[373048]
0
value is 373048
It's not ok
1
value is 14475
No_duplication
It's ok
14475
0
value is 8215
No_duplication
It's ok
8215
[14475, 8215]
0
value is 14475
It's not ok
1
value is 14302
No_duplication
It's ok
14302
0
value is 9034
No_duplication
It's ok
9034
[14302, 9034]
0
value is 14302
It's not ok
1
value is 1953
No_duplication
It's ok
1953
0
value is 1953
It's ok
1953
[1953]
0
value is 1953
It's not ok
1
value is 3090
No_duplication
It's ok
3090
0
value is 3090
It's ok
3090
[3090]
0
value is 3090
It's not ok
1
value is 1659
No_duplication
It's ok
1659
0
value is 1659
It's ok
1659
[1659]
0
value is 1659
It's not ok
1
value is 8043
No_duplication
I

In [18]:
FIT_search_ad = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Xb-8k7DlTNyQjJo3M5bMjw/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507/")
FIT_search_ad.set_tab(2)
FIT_search_ad.set_value([2,2])
FIT_search_ad.get([1,2])
FIT_search_iOS = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p8shpiJuS4S2PNncs3XiDQ/a10587421w80067081p88823264/")
FIT_search_iOS.set_tab(2)
FIT_search_iOS.set_value([2,2])
FIT_search_iOS.get([1,2])
FIT_search_Moweb = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ERjO4gwrTtm8N0QyMBVMSw/a10587421w80067081p89357101/")
FIT_search_Moweb.set_tab(2)
FIT_search_Moweb.set_value([2,2])
FIT_search_Moweb.get([1,2])

0
value is 1310792
It's not ok
1
value is 21587
No_duplication
It's ok
21587
0
value is 20225
No_duplication
It's ok
20225
[21587, 20225]
0
value is 21587
It's not ok
1
value is 21587
It's not ok
2
value is 14847
No_duplication
It's ok
14847
0
value is 14346
No_duplication
It's ok
14346
[14847, 14346]
0
value is 14847
It's not ok
1
value is 11040
No_duplication
It's ok
11040
0
value is 10468
No_duplication
It's ok
10468
[11040, 10468]
0
value is 11040
It's not ok
1
value is 10005
No_duplication
It's ok
10005
0
value is 9577
No_duplication
It's ok
9577
[10005, 9577]
0
value is 10005
It's not ok
1
value is 5477
No_duplication
It's ok
5477
0
value is 5105
No_duplication
It's ok
5105
[5477, 5105]
0
value is 5477
It's not ok
1
value is 4796
No_duplication
It's ok
4796
0
value is 4566
No_duplication
It's ok
4566
[4796, 4566]


In [77]:
pd.concat([And_6.df, And_7.df, iOS_6.df,iOS_7.df,Mo_6.df,PC_6.df])

,170522 - 170528
And 시스템 큐레이션 2. 주말특가 view,6743.0
And 시스템 큐레이션 2. 주말특가 클릭,580.0
And 카테고리 1. 여행홈 -> B.com(실시간 호텔),2218.0
And 카테고리 2. 국내호텔 배너클릭,20.0
And 카테고리 3. 해외호텔 메인 검색모듈 view,4129.0
And 카테고리 3. 해외호텔 메인 검색모듈 -> 이동,2966.0
And 시스템 큐레이션 1. 여행홈 -> B.com,835.0
And_ 매뉴얼큐레이션 - 1.큐레이션 딜view,3463.0
매뉴얼큐레이션 - 1.큐레이션 딜 view -> 이동,997.0
And_이벤트 페이지,8925.0


In [20]:
# #국내, 해외, 항공호텔, 전체카테고리
df1 = pd.concat([And_1.df, iOS_1.df, Mo_1.df, PC_1.df])

# #티몬 전체 카테고리
df2 = pd.concat([And_2.df, iOS_2.df, Mo_2.df, PC_2.df])

# #국내딜, 해외딜, 전체딜
#df3 = pd.concat([And_3.df, iOS_3.df, Mo_3.df, PC_3.df])

# #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
df4 = pd.concat([And_4.df, iOS_4.df, Mo_4.df, PC_4.df])

# #항공예약클릭 국내, 해외
df5 = pd.concat([And_5.df, iOS_5.df, Mo_5.df, PC_5.df])

# #부킹닷컴
df6 = pd.concat([And_6.df, And_7.df, iOS_6.df,iOS_7.df,Mo_6.df,PC_6.df])

# #홈 -> 딜상세(국내), 해외, 여행카테고리숏컷
df7 = pd.concat([And_8.df, iOS_8.df, Mo_7.df, PC_7.df])

# #홈 -> 콜렉션(국내), 해외
df8 = pd.concat([And_9.df, iOS_9.df])
# #여행홈 -> 카테고리, 항공검결
df9 = pd.concat([And_10.df, iOS_10.df])

# # 실시간 항공 국내/해외 구분
df10 = pd.concat([FIT_search_ad.df, FIT_search_iOS.df, FIT_search_Moweb.df])

In [35]:
df6

,170529 - 170604
And 시스템 큐레이션 2. 주말특가 view,7791.0
And 시스템 큐레이션 2. 주말특가 클릭,639.0
And 카테고리 1. 여행홈 -> B.com(실시간 호텔),2057.0
And 카테고리 2. 국내호텔 배너클릭,26.0
And 카테고리 3. 해외호텔 메인 검색모듈 view,4440.0
And 카테고리 3. 해외호텔 메인 검색모듈 -> 이동,3192.0
And 시스템 큐레이션 1. 여행홈 -> B.com,625.0
And_ 매뉴얼큐레이션 - 1.큐레이션 딜view,2658.0
매뉴얼큐레이션 - 1.큐레이션 딜 view -> 이동,756.0
And_이벤트 페이지,5586.0


In [36]:
df7

,170529 - 170604
And_홈화면,7555101.0
And_딜상세(국내),39833.0
And_딜상세(해외),8274.0
And_여행카테고리 숏컷,2163.0
iOS_홈화면,2571224.0
iOS_딜상세(국내),2859.0
iOS_딜상세(해외),1598.0
iOS_카테고리숏컷,13670.0
Moweb_홈화면,755056.0
Moweb_딜상세(국내),4524.0


In [24]:
# #B.com 카테고리/시스템 큐레이션
# And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507/")
# And_6.set_tab(5)
# And_6.set_value([2,1,1,2,1])
# And_6.set_date(7,3)
# And_6.get([1,2,3,4,5])

# #B.com 매뉴얼 큐레이션/이벤트
# And_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/qWyiPek-Tsqgkok-rZsIjA/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507%26_r.tabId%3D210/")
# And_7.set_tab(2)
# And_7.set_value([2,2])
# And_7.set_date(7,3)
# And_7.get([1,2])

# #B.com 카테고리/시스템 큐레이션
# iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/mzJgxll5SXqATSN_oHrUiQ/a10587421w80067081p88823264/")
# iOS_6.set_tab(5)
# iOS_6.set_value([2,1,1,2,1])
# iOS_6.set_date(7,3)
# iOS_6.get([1,2,3,4,5])

# #B.com 매뉴얼 큐레이션/이벤트
# iOS_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/tZRMJC4JRhm_X9Ql6C4KOg/a10587421w80067081p88823264/")
# iOS_7.set_tab(2)
# iOS_7.set_value([2,2])
# iOS_7.set_date(7,3)
# iOS_7.get([1,2])

# #부킹닷컴 큐레이션, 호텔메인, 이벤트
# Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/%3F_r.tabId%3D2101/")
# Mo_6.set_tab(3)
# Mo_6.set_value([2,2,2])
# Mo_6.set_date(7,3)
# Mo_6.get([1,2,3])

# # B.com 카테고리/시스템/매뉴얼큐레이션/이벤트
# PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_u.date00%3D20170501%26_u.date01%3D20170507%26_r.tabId%3D4292/")
# PC_6.set_tab(3)
# PC_6.set_value([1,2,2,1])
# PC_6.set_date(7,3)
# PC_6.get([1,2,3,4])

In [21]:
# #######월간 여행딜

# #국내, 해외, 여행전체 딜
# And_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/c9agY1srRx2PaD8JAkK0yw/a10587421w80067081p102782120/")
# And_3.set_tab(3)
# And_3.set_value([3,3,3])
# And_3.get([1,2,3])

# And_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# And_3.set_value([[3,3],[3,3],[3,3]])
# And_3.get([1,2,3])

# #국내딜, 해외딜, 전체딜
# iOS_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/A9m3wyZgQTmxJrO3lQ6UMw/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402%26_r.tabId%3D4646%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# iOS_3.set_tab(3)
# iOS_3.set_value([3,3,3])
# iOS_3.get([1,2,3])

# iOS_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# iOS_3.set_value([[3,3],[3,3],[3,3]])
# iOS_3.get([1,2,3])

# #여행딜 전체, 국내, 해외
# Mo_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/upJUz96ERVGUIE2WWd8Tgg/a10587421w80067081p89357101/")
# Mo_3.set_tab(3)
# Mo_3.set_value([3,3,3])
# Mo_3.get([1,2,3])

# Mo_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# Mo_3.set_value([[3,3],[3,3],[3,3]])
# Mo_3.get([1,2,3])

# # 여행 전체, 국내, 해외딜
# PC_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hRoEL4w4RCyYpcmhDbvgWw/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_3.set_tab(3)
# PC_3.set_value([3,3,3])
# PC_3.get([1,2,3])

# PC_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# PC_3.set_value([[3,3],[3,3],[3,3]])
# PC_3.get([1,2,3])

In [ ]:
#부킹닷컴 큐레이션, 호텔배너, 주말특가, 검색모듈
And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_6.set_tab(4)
And_6.set_value([2,1,2,2])
And_6.get([1,2,3,4])
#부킹닷컴 큐레이션, 호텔배너, 주말특가, 해외호텔검색모듈
iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/fiA5QbsjQAeJxkmaYUFJEQ/a10587421w80067081p88823264/")
iOS_6.set_tab(4)
iOS_6.set_value([2,1,2,2])
iOS_6.get([1,2,3,4])
#부킹닷컴 큐레이션, 호텔메인
Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/")
Mo_6.set_tab(2)
Mo_6.set_value([2,2])
Mo_6.get([1,2])
# 부킹닷컴 큐레이션, 해외호텔 카테고리, 주말특가
PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_6.set_tab(3)
PC_6.set_value([2,1,2])
PC_6.get([1,2,3])

In [14]:
# booking_df_User = pd.concat([And_6.df, iOS_6.df, Mo_6.df, PC_6.df])

In [49]:
# #######월간항공

# #항공검색 해외, 국내, 해외->호텔메인, 해외->국내항공메인
# And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5fyYOU-QTN6tQSwRlObkUw/a10587421w80067081p102782120/%3F_u.date00%3D20170101%26_u.date01%3D20170301%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# And_4.set_tab(4)
# And_4.set_value([2,2,[2,2],[2,2]])
# And_4.get([1,2,3,4])

# #항공예약 클릭 국내, 해외 
# And_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/_yDrAZzXTaeMOGlYkwqqbw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D%26_r.tabId%3D6028/")
# And_5.set_tab(2)
# And_5.set_value([1,1])
# And_5.get([1,2])

# #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
# iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/2K60xdJeT2isKsPe4tgpEQ/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170320/")
# iOS_4.set_tab(4)
# iOS_4.set_value([2,2,[2,2],[2,2]])
# iOS_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# iOS_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/btj6gUJmQ2-NZsSB1vB3tw/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170331%26_.useg%3D/")
# iOS_5.set_tab(2)
# iOS_5.set_value([1,1])
# iOS_5.get([1,2])

# #항공검색 해외, 국내, 해외->호텔, 해외->국내
# Mo_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/7aMTsZECQu2GVnxwp3IXRg/a10587421w80067081p89357101/")
# Mo_4.set_tab(4)
# Mo_4.set_value([2,2,[2,2],[2,2]])
# Mo_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# Mo_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Shs74NpmR1Wcy5isHt_ZGA/a10587421w80067081p89357101/")
# Mo_5.set_tab(2)
# Mo_5.set_value([1,1])
# Mo_5.get([1,2])

# # 항공검색 국내, 해외
# PC_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/KkWuVj5-SCOH7ddtrf4bOA/a10587421w80067081p84826225/")
# PC_4.set_tab(2)
# PC_4.set_value([2,2])
# PC_4.get([1,2])

# # 예약클릭 국내, 해외
# PC_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/YspUfeZLQ7KyhFzRxehRcQ/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_5.set_tab(2)
# PC_5.set_value([2,2])
# PC_5.get([1,2])

In [48]:
# pd.concat([And_4.df, And_5.df, iOS_4.df, iOS_5.df, Mo_4.df, Mo_5.df, PC_4.df, PC_5.df])